In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,431 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [36]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 
# to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
import pandas as pd
from pyspark.sql.functions import to_date
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).filter("total_votes>=20")
vine_pddf = vine_df.toPandas()
vine_pddf.dtypes

review_id            object
star_rating           int32
helpful_votes         int32
total_votes           int32
vine                 object
verified_purchase    object
dtype: object

In [44]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows 
# vwhere the number of helpful_votes divided by total_votes is equal to or greater than 50%.

vine_pddf['ratio'] = vine_pddf.helpful_votes / vine_pddf.total_votes #creating a new column with the ratio
vine_pddf_filtered = vine_pddf['ratio']>=0.5 #creating the filer boolean
vine_ready = vine_pddf[vine_pddf_filtered] #using the boolean, creating a filtered view of the table
vine_ready

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,ratio
0,R2243Y3OD8U6KQ,5,47,61,N,N,0.770492
1,R2TGT0CDTCAAHW,5,21,23,N,Y,0.913043
2,RX4D22YSXEF4P,1,37,38,N,Y,0.973684
3,R3FL2NTLFUSPTQ,5,33,37,N,N,0.891892
4,R3QTP3YNZXAPPF,3,23,24,N,Y,0.958333
...,...,...,...,...,...,...,...
16515,R2EJ952S3YK254,4,46,47,N,N,0.978723
16516,R6MDTUK723YPY,4,143,148,N,N,0.966216
16517,R10B4417NCK98O,5,24,24,N,N,1.000000
16518,R3V2A0GUZ3JAQ4,4,27,29,N,N,0.931034


In [48]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows 
# where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_y = vine_ready['vine']=='Y'
vine_yes = vine_ready[vine_y]
vine_yes.head(20)



,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,ratio
22,R1R9RU7JW0MFR2,4,20,23,Y,N,0.869565
45,R19EFYNN3W8Q07,5,26,32,Y,N,0.812500
183,R34DJ1R8AEU0SG,5,29,35,Y,N,0.828571
209,R25P5CXK5L9RHF,5,146,161,Y,N,0.906832
262,R2E9VZB3I4LSN5,5,55,59,Y,N,0.932203
386,RKYLHZL7EPELX,4,19,25,Y,N,0.760000
887,R1U13EKGQD3ZE6,5,22,25,Y,N,0.880000
956,RYW05F1MUEF01,5,87,102,Y,N,0.852941
1127,R2SW4NXNO7HZJ5,4,28,33,Y,N,0.848485
1180,R2016NFLSUR97Y,2,26,37,Y,N,0.702703


In [49]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_n = vine_ready['vine']=='N'
vine_no = vine_ready[vine_n]
vine_no.head(20)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,ratio
0,R2243Y3OD8U6KQ,5,47,61,N,N,0.770492
1,R2TGT0CDTCAAHW,5,21,23,N,Y,0.913043
2,RX4D22YSXEF4P,1,37,38,N,Y,0.973684
3,R3FL2NTLFUSPTQ,5,33,37,N,N,0.891892
4,R3QTP3YNZXAPPF,3,23,24,N,Y,0.958333
5,R36V6V42VN5AS5,5,34,37,N,Y,0.918919
6,R27LZWE27BJPOB,5,22,23,N,N,0.956522
9,R1P7GJ0IN2BRNH,5,37,37,N,Y,1.000000
10,R2R6JPF9KOD2HJ,5,19,20,N,Y,0.950000
12,R2J0ZZGFXKM8KR,2,21,22,N,Y,0.954545


In [50]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

# total number of paid reviews
num_paid = len(vine_yes)
num_paid


60

In [51]:
# total number of unpaid reviews
num_unpaid = len(vine_no)
num_unpaid

14477

In [52]:
# paid, number of 5-start reviews
five_star_paid = len(vine_yes[vine_yes["star_rating"]==5])
five_star_paid

34

In [53]:
# unpaid, number of 5-star review
five_star_unpaid = len(vine_no[vine_no["star_rating"]==5])
five_star_unpaid

8212

In [60]:
# percentage of 5-star reviews , for paid
fivestar_ratio_paid = five_star_paid / num_paid
percentage_paid = "{:.0%}".format(fivestar_ratio_paid)
percentage_paid

'57%'

In [61]:
# percentage of 5-star reviews , for unpaid
fivestar_ratio_unpaid = five_star_unpaid / num_unpaid
percentage_unpaid = "{:.0%}".format(fivestar_ratio_unpaid)
percentage_unpaid

'57%'